In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
import pandas as pd
import os
import xgboost as xgb

In [ ]:
#load all dfs form a folder and merge them on ID
""" path = "/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/"
for i, file in enumerate(os.listdir(path)):
    if i == 0:
        df = pd.read_csv(path + file)
    else:
        df = pd.merge(df, pd.read_csv(path + file), on='ID') """
df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/deconfounded_but_age/aseg.volume_aparc.volume_aparc.thickness.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/00_NAKO/00_data/age_label/all_ages.csv")

label_df = label_df[['ID', 'label_age_group']]

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
for col in label_df.columns:
    print(col)

In [ ]:
#merge the dataframes on ID
merged_df = pd.merge(df, label_df, on='ID', how='inner')

In [ ]:
merged_df["label_age_group"].value_counts()
merged_df.dropna(inplace=True)
merged_df = merged_df[:5000]

#drop specific labels for label_age_group
merged_df = merged_df[merged_df.label_age_group != 10]

In [ ]:
y = merged_df["label_age_group"]
X = merged_df.drop(["ID", "label_age_group"], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
import numpy as np
from sklearn.metrics import balanced_accuracy_score, classification_report


# Initialize the classifier
clf = TabPFNClassifier()

# Fit the model
clf.fit(X_train, y_train)

# Predict probabilities
y_pred_proba = clf.predict_proba(X_test)

y_pred = clf.predict(X_test)
    






In [ ]:
y_pred_proba

In [ ]:
y_pred_proba_control

In [ ]:
y_test.shape

In [ ]:
# ROC AUC for multiclass using 'macro' average
# ROC AUC for multiclass using 'macro' average
auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_test))
random_y_test = np.random.randint(0, n_classes, size=y_test.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred)

In [ ]:
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)

In [ ]:
#use xgboost
xg_clf = xgb.XGBClassifier()
xg_clf.fit(X_train, y_train)
y_pred = xg_clf.predict(X_test)
y_pred_proba = xg_clf.predict_proba(X_test)

In [ ]:
# ROC AUC for multiclass using 'macro' average
auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_test, y_pred)
balanced_acc = balanced_accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_test))
random_y_test = np.random.randint(0, n_classes, size=y_test.shape)
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)

In [ ]:
#verify on onther dataset
df_control = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/deconfounded_but_age/aseg.volume_aparc.thickness_aparc.volume.csv")
label_df = pd.read_csv("/zi/home/esra.lenz/Documents/00_HITKIP/09_TABPFN/01_Validation_data_set/00_data/age_label/all_ages.csv")

label_df = label_df[['ID', 'label_age_group']]
df_control = df_control[df.columns]

In [ ]:
#get the label and the data
merged_df_control = pd.merge(df_control, label_df, on='ID', how='inner')
merged_df_control.dropna(inplace=True)

X = merged_df_control.drop(["ID", "label_age_group"], axis=1)
y = merged_df_control["label_age_group"]

In [ ]:
#traintest split but just getting train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=None, random_state=42, stratify=y)

In [ ]:
len(X_train)

In [ ]:
len(y_train)

In [ ]:
#check that the colum order is the same for df and control give back false if not so
print(all(X.columns == X_train.columns))

In [ ]:
y_pred_control = clf.predict(X_train)
y_pred_proba_control = clf.predict_proba(X_train)

In [ ]:
print("y_score shape:", len(y_train))
print("y_true shape:", y_pred_proba_control.shape)
print("y_pred shape:", y_pred_control.shape)

In [ ]:
#ROC
#auc = roc_auc_score(y_control, y_pred_proba_control, multi_class='ovr', average='macro')
auc_control = roc_auc_score(y_train, y_pred_proba_control, multi_class='ovr', average='macro')
acc = accuracy_score(y_train, y_pred_control)
balanced_acc = balanced_accuracy_score(y_train, y_pred_control)
report = classification_report(y_train, y_pred_control)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_train))
random_y_test = np.random.randint(0, n_classes, size=y_train.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred_control)

#print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)


In [ ]:
# ROC AUC for multiclass using 'macro' average
#auc = roc_auc_score(y_control, y_pred_proba, multi_class='ovr', average='macro')

acc = accuracy_score(y_control, y_pred)
balanced_acc = balanced_accuracy_score(y_control, y_pred)
report = classification_report(y_control, y_pred)

# For random comparison, use the number of classes instead of 2
n_classes = len(np.unique(y_control))
random_y_test = np.random.randint(0, n_classes, size=y_control.shape)
random_balanced_acc = balanced_accuracy_score(random_y_test, y_pred)

In [ ]:
print(f"ROC AUC: {auc}")
print(f"Accuracy: {acc}")
print(f"Balanced Accuracy: {balanced_acc}")
print(f"Random Balanced Accuracy: {random_balanced_acc}")
print(report)